In [1]:
%pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/repo.html
%pip install labml-nn

Looking in links: https://data.dgl.ai/wheels/torch-2.3/repo.html
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pydantic


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import dgl
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.sparse as sp
from scipy.sparse.linalg import expm
import numpy as np

c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()
c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class LinearNeuralNetwork(nn.Module):
    def __init__(self, nfeat, nclass, bias=True):
        super(LinearNeuralNetwork, self).__init__()
        self.W = nn.Linear(nfeat, nclass, bias=bias)

    def forward(self, x):
        return self.W(x)


def symmetric_normalize_adjacency(graph):
    """Symmetric normalize graph adjacency matrix."""
    indices = torch.stack(graph.edges())
    n = graph.num_nodes()
    adj = dglsp.spmatrix(indices, shape=(n, n))
    deg_invsqrt = dglsp.diag(adj.sum(0)) ** -0.5
    return deg_invsqrt @ adj @ deg_invsqrt


def model_test(model, embeds):
    model.eval()
    with torch.no_grad():
        output = model(embeds)
        pred = output.argmax(dim=-1)
        test_mask, tv_mask = model.test_mask, model.tv_mask
        loss_tv = F.mse_loss(output[tv_mask], model.label_one_hot[tv_mask])
    accs = []
    for mask in [tv_mask, test_mask]:
        accs.append(
            float((pred[mask] == model.label[mask]).sum() / mask.sum()))
    return loss_tv.item(), accs[0], accs[1], pred

In [5]:
class OGC(nn.Module):
    def __init__(self, graph):
        super(OGC, self).__init__()
        self.linear_clf = LinearNeuralNetwork(
            nfeat=graph.ndata["feat"].shape[1],
            nclass=graph.ndata["label"].max().item() + 1,
            bias=False,
        )

        self.label = graph.ndata["label"]
        self.label_one_hot = F.one_hot(graph.ndata["label"]).float()
        # LIM trick, else use both train and val set to construct this matrix.
        self.label_idx_mat = dglsp.diag(graph.ndata["train_mask"]).float()

        self.test_mask = graph.ndata["test_mask"]
        self.tv_mask = graph.ndata["train_mask"] + graph.ndata["val_mask"]

    def forward(self, x):
        return self.linear_clf(x)

    def update_embeds(self, embeds, lazy_adj, args):
        """Update classifier's weight by training a linear supervised model."""
        pred_label = self(embeds).data
        clf_weight = self.linear_clf.W.weight.data

        # Update the smoothness loss via LGC.
        embeds = dglsp.spmm(lazy_adj, embeds)

        # Update the supervised loss via SEB.
        deriv_sup = 2 * dglsp.matmul(
            dglsp.spmm(self.label_idx_mat, -self.label_one_hot + pred_label),
            clf_weight,
        )
        embeds = embeds - args.lr_sup * deriv_sup

        args.lr_sup = args.lr_sup * args.decline
        return embeds

In [6]:
import argparse
import time

import dgl.sparse as dglsp

import torch.nn.functional as F
import torch.optim as optim
from dgl import AddSelfLoop
from dgl.data import CiteseerGraphDataset

parser = argparse.ArgumentParser()
args = parser.parse_args([])
args.dataset = "citeseer"
args.decline = 0.9
args.lr_sup = 0.001
args.lr_clf = 0.5
args.beta = 0.1
args.max_sim_rate = 0.995
args.max_patience = 2
args.device = torch.device("cpu")

In [7]:
def train(model, embeds, lazy_adj, args):
    patience = 0
    _, _, last_acc, last_output = model_test(model, embeds)

    tv_mask = model.tv_mask
    optimizer = optim.SGD(model.parameters(), lr=args.lr_clf)

    for i in range(64):
        model.train()
        output = model(embeds)
        loss_tv = F.mse_loss(
            output[tv_mask], model.label_one_hot[tv_mask], reduction="sum"
        )
        optimizer.zero_grad()
        loss_tv.backward()
        optimizer.step()

        # Updating node embeds by LGC and SEB jointly.
        embeds = model.update_embeds(embeds, lazy_adj, args)

        loss_tv, acc_tv, acc_test, pred = model_test(model, embeds)
        print(
            "epoch {} loss_tv {:.4f} acc_tv {:.4f} acc_test {:.4f}".format(
                i + 1, loss_tv, acc_tv, acc_test
            )
        )

        sim_rate = float(int((pred == last_output).sum()) / int(pred.shape[0]))
        if sim_rate > args.max_sim_rate:
            patience += 1
            if patience > args.max_patience:
                break
        last_acc = acc_test
        last_output = pred
    return last_acc

In [8]:
!nvidia-smi

Sat Feb 22 10:38:37 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P0              12W /  50W |     99MiB /  4096MiB |     19%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
transform = AddSelfLoop()
data = CiteseerGraphDataset(transform=transform)
graph = data[0].to(args.device)

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [10]:
graph.has_edges_between([0], [1])

tensor([False])

In [11]:
def get_heat(adj_matrix: sp.csr_matrix, t: float = 5.0, k: int = 128):
    """Adapted from GDC's diffusion.py - computes heat kernel diffusion matrix"""
    n = adj_matrix.shape[0]
    L = sp.eye(n, format="csr") - adj_matrix  # Unnormalized Laplacian
    heat = sp.eye(n, format="csr")
    L_power = sp.eye(n, format="csr")
    fact_t = 1.0

    for i in range(1, k):
        L_power = L_power.dot(L)
        fact_t *= t / i
        heat -= fact_t * L_power

    return heat

# 2. Compute Heat Kernel PageRank score by GDC


def compute_gdc_hkpr(graph, t=5.0, k=128):
    """Compute HKPR scores using GDC's approximation"""
    # Get symmetric normalized adjacency in SciPy format
    indices = torch.stack(graph.edges())
    n = graph.num_nodes()
    adj = sp.coo_matrix(
        (
            np.ones(indices.shape[1]),  # Data (all ones for unweighted edges)
            (indices[0].numpy(), indices[1].numpy())  # Row and column indices
        ),
        shape=(n, n)
    ).tocsr()  # Convert to CSR format for efficient computations

    # Compute heat kernel matrix
    heat = get_heat(adj, t=t, k=k)

    # Compute HKPR scores (uniform personalization)
    s = np.ones(n) / n  # Uniform starting vector
    hkpr_scores = heat.dot(s)  # Matrix-vector multiplication

    return torch.tensor(hkpr_scores, dtype=torch.float32)

In [32]:
# Compute HKPR scores using GDC's method
hkpr_scores = compute_gdc_hkpr(graph, t=3.0, k=64).to(args.device)
hkpr_scores /= hkpr_scores.sum()  # Normalize




print(hkpr_scores)

tensor([1.5520e-16, 1.1552e-04, 1.5520e-16,  ..., 5.8693e-04, 1.5520e-16,
        1.5806e-08])


In [33]:
# Create diagonal matrix from HKPR scores
hkpr_diag = dglsp.diag(hkpr_scores)

# Symmetric normalized adjacency (DGL format)
adj = symmetric_normalize_adjacency(graph)

# Build lazy adjacency matrix with HKPR teleportation
lazy_adj = dglsp.add((1 - args.beta) * hkpr_diag,
                     args.beta * adj).to(args.device)

In [34]:
# Train the model
model = OGC(graph).to(args.device)
print(f"Model Parameters: {sum(p.numel() for p in model.parameters())}")

Model Parameters: 22218


In [51]:
features = graph.ndata["feat"]
start_time = time.time()
res = train(model, features, lazy_adj, args)
time_tot = time.time() - start_time

print(f"Test Acc: {res:.4f}")
print(f"Total Time: {time_tot:.4f}")

epoch 1 loss_tv 0.1484 acc_tv 0.8758 acc_test 0.7660
epoch 2 loss_tv 0.1647 acc_tv 0.8694 acc_test 0.7740
epoch 3 loss_tv 0.1665 acc_tv 0.8581 acc_test 0.7680
epoch 4 loss_tv 0.1666 acc_tv 0.8468 acc_test 0.7700
epoch 5 loss_tv 0.1667 acc_tv 0.8387 acc_test 0.7650
epoch 6 loss_tv 0.1667 acc_tv 0.8355 acc_test 0.7620
epoch 7 loss_tv 0.1667 acc_tv 0.8306 acc_test 0.7600
epoch 8 loss_tv 0.1667 acc_tv 0.8226 acc_test 0.7570
epoch 9 loss_tv 0.1667 acc_tv 0.8177 acc_test 0.7560
epoch 10 loss_tv 0.1667 acc_tv 0.8145 acc_test 0.7540
epoch 11 loss_tv 0.1667 acc_tv 0.8097 acc_test 0.7530
epoch 12 loss_tv 0.1667 acc_tv 0.8097 acc_test 0.7520
epoch 13 loss_tv 0.1667 acc_tv 0.8081 acc_test 0.7520
Test Acc: 0.7520
Total Time: 1.7774
